In [34]:
import pandas as pd
import numpy as np


df = pd.read_csv("dataset.csv")


df = df.drop("Unnamed: 0", axis=1) # drop index



In [35]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer, StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
string_cols = df.select_dtypes(include=['object', 'string']).columns.tolist()

text_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('one_hot', OneHotEncoder(sparse_output=False))
])

number_pipeline = Pipeline([
    ('imputer', SimpleImputer()),
    ('scaler', StandardScaler())
])


ct = ColumnTransformer([
    ("text_pipe", text_pipeline, string_cols),
    ("number_pipe", number_pipeline, numeric_cols)
])


df = pd.DataFrame(ct.fit_transform(df), columns=ct.get_feature_names_out())
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1935 entries, 0 to 1934
Data columns (total 27 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   text_pipe__blue_no                     1935 non-null   float64
 1   text_pipe__blue_yes                    1935 non-null   float64
 2   text_pipe__four_g_no                   1935 non-null   float64
 3   text_pipe__four_g_yes                  1935 non-null   float64
 4   text_pipe__wifi_no_wifi                1935 non-null   float64
 5   text_pipe__wifi_wifi                   1935 non-null   float64
 6   text_pipe__price_range_high cost       1935 non-null   float64
 7   text_pipe__price_range_low cost        1935 non-null   float64
 8   text_pipe__price_range_medium cost     1935 non-null   float64
 9   text_pipe__price_range_very high cost  1935 non-null   float64
 10  number_pipe__battery_power             1935 non-null   float64
 11  numb

In [36]:
from sklearn.model_selection import train_test_split

y_titles = np.array(["text_pipe__price_range_high cost", "text_pipe__price_range_low cost", "text_pipe__price_range_medium cost", "text_pipe__price_range_very high cost"])

X = df.drop(columns=y_titles, axis=1)
y = df[y_titles]

X_train, X_to_split, y_train, y_to_split = train_test_split(X, y, test_size=0.66, random_state=42)
X_validate, X_test, y_validate, y_test = train_test_split(X_to_split, y_to_split, test_size=0.5, random_state=42)


In [ ]:
import keras
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

input_count = 23
output_count = 4
keras.utils.set_random_seed(42)

model = keras.Sequential([
    keras.layers.Input((input_count,)),  # If we write (n_input_features), it will simplify to int. If add a comma, it will be a single-element tuple
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(output_count)
])

model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_20 (Dense)                │ (None, 100)            │         2,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_21 (Dense)                │ (None, 100)            │        10,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ (None, 100)            │        10,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_23 (Dense)                │ (None, 23)             │         2,323 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 24,923 (97.36 KB)

 Trainable params: 24,923 (97.36 KB)

 Non-trainable params: 0 (0.00 B)

In [38]:
optimizer = keras.optimizers.Adam(learning_rate=0.1)
model.compile(optimizer=optimizer, loss="mse")
initial_weights = model.get_weights()

In [39]:
history = model.fit(X_train, y_train, epochs=100, validation_data=[X_test, y_test])

Epoch 1/100


ValueError: Dimensions must be equal, but are 4 and 23 for '{{node compile_loss/mse/sub}} = Sub[T=DT_FLOAT](data_1, sequential_5_1/dense_23_1/BiasAdd)' with input shapes: [?,4], [?,23].

In [ ]:
model_predictions = model.predict(X_test)
mean_absolute_error(y_test, model_predictions)